# 処理の関数化

In [1]:
import ccxt
from pprint import pprint

In [ ]:
import requests
from datetime import datetime
import time

# データ取得(10秒ごと)
def get_price(min):
    response = requests.get('https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc', params = {'periods' : min})
    response = response.json()
    data = response['result'][str(min)][-2]

    close_time = data[0]
    open_price = data[1]
    close_price = data[4]

    return close_time, open_price, close_price

# 表示
def print_price(close_time, open_price, close_price):
    print('時間: ' + datetime.fromtimestamp(close_time).strftime('%Y/%m/%d %H:%M')
        + ' 始値: ' + str(open_price)
        + ' 終値: ' + str(close_price)
    )

last_time = 0
while True:
    close_time, open_price, close_price = get_price(60)

    if close_time != last_time:
        last_time = close_time

        print_price(close_time, open_price, close_price)

    time.aleep(10)



## entry & exitの自動化

In [ ]:
# flg
flg = {
    'buy_signal': False,    #　buyシグナルの状況
    'order': False,          #　サーバに注文があるかどうか
    'position': False       # position保有の有無
}

# entry
def buy_signal(data, last_data, flg):
    # if buyの条件:
        # buy注文のコード
    # else:
    #     flg['buy_signal'] = False
    return flg

# 約定チェック（全体のwhileループで、10秒おきに約定状況を確認）
def check_order(flg):
    # if 注文が約定済み:
        flg['order'] = False
        flg['position'] = True # ここがTrueになってはじめて、close_position関数が動作する
    return flg

# 手仕舞い
# flg['position'] = True であれば、close_position関数が呼ばれる
def close_position(date, last_data, flg):
    if data['close_price'] < last_data[['close_price']]:
        print("前回の終値を下回ったので" + str(data["close_price"]) + "で決済")
        flg['position'] = False
    return flg



## 全体の管理 

In [ ]:
# 実際の環境では10秒間隔でループする
last_data = get_price(60, 0)
print_price(last_data)

flg = {
    'buy_signal': False,
    'sell_signal': False,
    'order': False,
    'position': False
}

while True:
    # まず最優先で「未約定注文がサーバーにあるかどうか」確認
    if flg['order']:    # 未約定フラグがTrueなら
        flg = check_order(flg)   # サーバにチェックに行く 一定時間経っていればキャンセル

    data = get_price(60, i) # 1分足のデータ取得　これは1分ごとのデータ取得という意味ではない
    if data['close_time'] != last_data['close_time']:
        print_price(data)

        if flg['position']:
            # ポジションがあれば、手仕舞条件を満たしているか確認。close or stay。
            flg = close_position(data, last_data, flg)
        else:
            # ポジションがなければ、buy_signalがTrueかどうか確認。 Trueなら注文。
            flg = buy_signal(data, last_data, flg)

        last_data['close_time'] = data['close_time']
        last_data['open_price'] = data['open_price']
        last_data['close_price'] = data['close_price']
    
    time.sleep(10)

## テスト検証のためのコード

In [ ]:
import requests
from datetime import datetime
import time

response = requests.get("https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc",
            params = { "periods" : 60 })


def get_price(min, i):
    data = response.json()
    return{
        'clone_time': data['result'][str(min)][i][0],
        'open_price': data['result'][str(min)][i][1],
        'high_price': data['result'][str(min)][i][2],
        'low_price': data['result'][str(min)][i][3],
        'close_price': data['result'][str(min)][i][4],
    }

def print_price(data):
    print('時間: ' + datetime.fromtimestamp(data['close_time']).strftime('%Y%m%d %H:%M') + '始値: ' + str(data['open_price']) + '終値: ' + str(data['close_price']))

def check_candle(data):
    # candleの基準
    op = data['open_price']
    cl = data['close_pricee']
    hi = data['high_price']
    lo = data['low_price']
    realbody_rate = abs(cl - op) / (hi - lo) # 全体に対する実体サイズの割合
    increase_rate = cl / op - 1              # 上昇率

    if cl < op : return False                     # 陽線
    elif increase_rate < 0.0005 : return False    # 実態の割合条件
    elif realbody_rate < 0.5 : return False       # 上昇率の条件
    else : return True

# highとlowが切り上がってるかどうか
def check_ascend(data, last_data):
    if data['open_price'] > last_data['open_price'] and data['close_price'] > last_data['close_price']:
        return True
    else:
         return False

# buy_signalでbuy実行
def buy_signal(data, last_data, flg):
    # if buyの条件:
        # buy実行のコード
        flg['buy_signal'] = True # buyの条件だけで、buy実行ならこのフラグは不要
        flg['order'] = True
    # else:
        flg['buy_signal'] = False # buyの条件だけで、buy実行ならこのフラグは不要
    pass # コメントによるwarnig回避のためのpass 運用時には消す
    # return flg

# 手仕舞い
def close_position(data, last_data, flg):
    if data['close_price'] < last_data['close_price']:
        print('前回の終値を下回ったので' + str(data['close_price']) + 'で決済')
        flg['position'] = False
    return flg 

# 約定確認
def check_order(flg):
    # if 注文確認、約定済なら以下のコードでflg変更
    # 未約定なら、一定時間でキャンセル

    # 約定済みの時のコード
        flg['order'] = False
        flg['position'] = True
    return flg



# 以下、メインコード（全体の管理）
# テスト検証ではsleep(10)で待つ必要はない
last_data = get_price(60, 0)
print_price(last_data)

flg = {
    'buy_signal': False,
    'sell_signal': False,
    'order': False,
    'position': False
}

i = 1
while i < 500: #　500本のローソク足をループ
    # 最優先でサーバー上注文の有無を確認
    if flg['order']: # 未約定フラグがTrueなら、
        flg = check_order(flg) # サーバ上の注文状況確認

    data = get_price(60, i)
    if data['close_time'] != last_data['close_time']: # candleの更新があれば
        print_price(data)
        
        # ポジションを持っていれば手仕舞いのコードを実行
        if flg['position']:
            flg = close_position(data, last_data, flg)
        # ポジションがなければ、buy_signalでbuyするかどうか判断し、条件が合えばbuy_signalの中でbuyが実行される
        else:
            flg = buy_signal(data, last_data, flg)

        last_data['close_time'] = data['close_time']
        last_data['open_price'] = data['open_price']
        last_data['close_price'] = data['close_price']
        i += 1

    # テスト環境では待ち時間は０
    time.sleep(0)

## 試作bot  
例外処理も組み込み済  
buy,sellの条件はまだ未作成 この条件を機械学習で処理する

In [ ]:
import requests
from datetime import datetime
import time
import ccxt

bitflyer = ccxt.bitflyer()
bitflyer.apiKey = ''
bitflyer.secret = ''

# データ取得
def get_price(min, i):
    while True:
        try:
            response = requests.get("https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc", params = { "periods" : 60 })
            data = response.json()
            return {
                'close_time': data['result'][str(min)][i][0],
                'open_price': data['result'][str(min)][i][1],
                'high_price': data['result'][str(min)][i][2],
                'low_price' : data['result'][str(min)][i][3],
                'close_price': data['result'][str(min)][i][4]   
            }
            # returnしてるのでbreakはいらない
        except requests.exceptions.RequestException as e:
            print('Cryptowatchの価格取得でエラー発生: ', e)
            print('10秒待機してやり直します')
            time.sleep(10)

def print_price(data):
    print( "時間: " + datetime.fromtimestamp(data["close_time"]).strftime('%Y/%m/%d %H:%M') + " 始値: " + str(data["open_price"]) + " 終値: " + str(data["close_price"]) )

# candleの基準の評価
def check_cnadle(data, side):
    op = data['open_price']
    cl = data['close_pricee']
    hi = data['high_price']
    lo = data['low_price']
    realbody_rate = abs(cl - op) / (hi - lo)
    increase_rate = cl / op - 1

    if side == 'buy':
        if cl < op: return False
        elif increase_rate < 0.0003: return False
        elif realbody_rate < 0.5: return Falsee
        else: return True
    
    if side == 'sell':
        if cl > op: return False
        elif increase_rate > -0.0003: False
        elif realbody_rate < 0.5: return False
        else: return True

# open,closeが共に切り上げてるか確認
def check_ascend(data, last_data):
    if data['open_price'] > last_data['open_price'] \
        and data['close_price'] > last_data['close_price']:
        return True
    else:
        return False

# open,closeが共に切り下げてるか確認
def check_descend(data, last_data):
    if data['open_price'] < last_data['open_price'] \
        and data['close_price'] < last_data['close_price']:
        return True
    else:
        return False

# buy_signalでbuy実行
# ここのbuy,sell条件が重要　まだ作ってない
def buy_signal(data, last_data, flg):
    # if buyの条件:
        # buy実行のコード
        flg['buy_signal'] = True # sellの条件だけで、sell実行ならこのフラグは不要
        try:
            order = bitflyer.create_order(
                symbol = 'BTC/JPY',
                type = 'limit' # 指値
                side = 'buy',
                price = data['close_price'], 
                amount = '0.001', # buy枚数
                params = {'product_code': 'FX_BTC_FX'} # テストは現物の方が売買最小単位が小さくて安全
            )
            flg['order']['exist'] = True # 注文済みかどうかのフラグ　約定済かどうかはメ、インコードでapi経由で問い合わせる
            flg['order']['side'] = 'BUY'
            time.sleep(30)
        except ccxt.BaseError as e:
            print('Bitflyerの注文でAPIエラーが発生', e)
            print('注文が失敗しました')
    # else:
        flg['buy_dignal'] = False # buyの条件だけで、buy実行ならこのフラグは不要
    # pass # コメントによるwarnig回避のためのpass 運用時には消す
    return flg

# sell_signalでsell実行
def sell_signal(data, last_data, flg):
    # if sellの条件:
        # sellの実行コード
        flg['sell_signal'] = True # sellの条件だけで、sell実行ならこのフラグは不要
        try:
            order = bitflyer.create_oder(
                symbol = 'BTC/JPY',
                type = 'limit',
                side = 'sell',
                price = data['close_price'],
                amount = '0.001', # sell枚数
                params = {'product_code': 'FX_BTC_FX'} # テストは現物の方が売買最小単位が小さくて安全
            )
            flg['order']['exist'] = True
            flg['order']['side'] = 'SELL'
            time.sleep(30)
        except ccxt.BaseError as e:
            print('Bitflyerの注文でAPIエラーが発生', e)
            print('注文が失敗しました')
    # else:
        flg['sell_signal'] = False # sellの条件だけで、sell実行ならこのフラグは不要
    # pass # コメントによるwarnig回避のためのpass 運用時には消す
    return flg    

#  手仕舞いのsignalでcslose
def close_position(data, last_data, flg):
    if flg['position']['side'] == 'BUY': #buy positionの時の手仕舞いコード
        if data['close_price'] < last_data['close_price']:
            print("前回の終値を下回ったので" + str(data["close_price"]) + "あたりで成行で決済します")
            while True:
                try:
                    order = bitflyer.create_order(
                        symbol = 'BTC/JPY',
                        type = 'market',
                        side = 'sell',
                        amount = '0.001',
                        params = {'product_code': 'FX_BTC_FX'}
                    )
                    flg['position']['exist'] = False
                    time.sleep(30)
                    break
                except ccxt.BaseError as e:
                    print('BitflyerのAPIでエラー発生', e)
                    print('注文の通信が失敗しました。30秒後に再トライします')
                    time.sleep(30)

    if flg['position']['side'] == 'SELL': #sell positionの時の手仕舞いコード
        if data['close_price'] > last_data['close_price']:
            print("前回の終値を上回ったので" + str(data["close_price"]) + "あたりで成行で決済します")
            while True:
                try:
                    order = bitflyer.create_order(
                        symbol = 'BTC/JPY',
                        type = 'market',
                        side = 'buy',
                        amount = '0.001',
                        params = {'product_code': 'FX_BTC_FX'}
                    )
                    flg['position']['exist'] = False
                    time.sleep(30)
                    break
                except ccxt.BaseError as e:
                    print('BitflyerのAPIでエラー発生', e)
                    print('注文の通信が失敗しました。30秒後に再トライします')
                    time.sleep(30)


# サーバ上の注文が約定したかどうかのチェック
# *** position == True and orders == True で複数のポジションを持っているときは、全体のループで管理。このチェックを繰り返す
def check_order(flg):
    try:
        position = bitflyer.private_get_getpositions(params={'product_code': 'FX_BTC_FX'})
        orders = bitflyer.fetch_open_orders(
            symbol = "BTC/JPY",
            params = { "product_code" : "FX_BTC_JPY" }
        )
    except ccxt.BaseError as e:
        print('BitflyerのAPIで問題発生: ', e)
    else: # try節が正常終了した後
        if position:
            print('注文が成立しました') 
            flg['order']['exist'] = False
            flg['order']['count'] = 0
            flg['position']['exist'] = True
            flg['position']['side'] = flg['order']['side']
        else:
            if orders:
                print('まだ未約定の注文があります')
                for o in orders:
                    print(p['id'])
                flg['order']['count'] += 1 # 1回のwhileループごとにインクリメント
                if flg['order']['count'] > 6: # 1分間（count==6で60秒経過）約定しないときは
                    flg = cancel_order(orders, flg)
            else:
                print('注文が遅延しているようです') # position == False and orders == False のとき
    return flg

# キャンセルのコード
# 利確のルールはない 上がり続けている間は保持して、一度下がった段階で決済する
def cancel_order(orders, flg):
    try:
        for o in orders:
            bitflyer.cancel_order(
                symbol = 'BTC/JPY',
                id = o['id'],
                params = {'product_code': 'FX_BTC_FX'}
            )
        print('未約定注文をキャンセルしました')
        flg['order']['count'] = 0
        flg['order']['exist'] = False
        
        time.sleep(20) # レイテンシーのためにキャンセルが間に合わなかった時のためのチェック
        position = bitflyer.private_get_positions(params = {'product_code': 'FX_BTC_FX'})
        if not position:
            print('現在、未決済建玉はありません')
        else:
            print('まだ、未決済建玉があります')
            flg['position']['exist'] = True # このフラグをTrueにする処理を忘れると、未決済建玉が溜まっていく
            flg['position']['side'] = positions[0]['side']
    except ccxt.BaseError as e:
        print('BitflyerのAPIで問題発生: ', e)
    finally:
        return flg

# メインコード
last_data = get_price(60, -2)
print_price(last_data)
time_sleep(10)

flg = {
    'buy_signal': False,
    'sell_signal': False,
    'order': {
        'exist': True,
        'side': '',
        'count': 0,
    },
    'position': {
        'exist': False,
        'side': ''
    }
}

while True:
    # まず、注文状況を確認。注文があるなら、
    if flg['order']['exist']: 
        flg = check_order(flg)

    data = get_price(60, -2) 
    if data['clsoe_time'] != last_data['close_time']: # candleが更新されたら
        print_price(data)
        if flg['position']['exist']:
            flg = close_position(data, last_data, flg) # これでロスカットを保証
            # *** 利確のルールはない 上がり続けている間は保持して、一度下がった段階で決済する
        else:
            flg = buy_signal(data, last_data, flg) # ここで注文発注まで行う
            flg = sell_signal(data, last_data, flg)
        # dataの更新
        last_data['close_time'] = data['close_time']
        last_data['open_price'] = data['open_price']
        last_data['close_price'] = data['close_price']
    time.sleep(10)